In [1]:
import json
import os
import glob
import pandas as pd
import time
import xmltodict
import forecastio
import datetime
from tqdm import tqdm
pd.set_option('display.max_columns', 50)

In [ ]:
### Single run through PGA Hole Parse for a tournament

os.chdir('/home/valesco/Datasets/PGA_Data/PGA/')

attempt_ls = ['005_2015']

tournaments = os.walk(os.getcwd())

tourny_dir = []

for t in tournaments:
    if 'PlayerScoreCards' not in t[0]:
        tourny_dir.append(t[0])
        
tourny_dir = tourny_dir[1:]
succeed_ls = []
for i, tourny in enumerate(attempt_ls):

    tourny_path = '/home/valesco/Datasets/PGA_Data/PGA/' + tourny
    os.chdir(tourny_path)

    # return tournament_code plus year
    tourny_prefix = tourny_path.replace('/home/valesco/Datasets/PGA_Data/PGA/','')


    # load course json file
    course_json = open(tourny_prefix + '_course.json')
    course_data = json.load(course_json)

    # load leaderboard json file
    leaderboard_json = open(tourny_prefix + '_leaderboard-v2.json')
    leaderboard_data = json.load(leaderboard_json)

    # load money json file
    money_json = open(tourny_prefix + '_money.json')
    money_data = json.load(money_json)

    # load player stats json file
    stats_json = open(tourny_prefix + '_player_stats.json')
    stats_data = json.load(stats_json)

    # load playoff json file
    playoff_json = open(tourny_prefix + '_playoff_info.json')
    playoff_data = json.load(playoff_json)

    # load teetimes json file
    tee_json = open(tourny_prefix + '_teetimes.json')
    tee_data = json.load(tee_json)

    # load tournsum json file
    tournsum_json = open(tourny_prefix + '_tournsum.json')
    tournsum_data = json.load(tournsum_json)

    # load course location file
    course_loc_json = open('/home/valesco/Datasets/PGA_Data/course_locations.json')
    course_loc_data = json.load(course_loc_json)

    # load player_id to player_name

    #Turn setup.xml to ordered dict to retrieve course grass type
    with open(tourny_prefix + '_setup.xml') as fd:
        setup_data = xmltodict.parse(fd.read())

    setup2_json = open(tourny_prefix + '_setup.json')
    setup2_data = json.load(setup2_json)

    # PlayerScoreCards Directory
    score_card_path = tourny_path + '/PlayerScoreCards/'

    tournament_dict = {}

    #Get tournament code, tourny, and course name
    tourny_code = tourny_prefix.split('_')[0]
    tourny_name = tee_data['tournament']['TournamentName']


    tournament_dict[tourny_code] = {'tournament_name': tourny_name}

    #Count number of courses played in tournament and dictate for json to use course or courses
    num_courses = len(course_data['courses'])

    if num_courses > 1:
        course_courses = 'courses'
    else:
        course_courses = 'course'

    #make dict of course info for each course in tournament
    tourny_course_dict = {}
    for e, course in enumerate(course_data['courses']):
        tourny_course_name = course_data['courses'][e]['name']
        tourny_course_num = course_data['courses'][e]['number']
        tourny_par_front = tee_data['tournament']['rounds'][0]['courses'][e]['Front']
        tourny_par_back = tee_data['tournament']['rounds'][0]['courses'][e]['Back']
        tourny_yardage = course_data['courses'][e]['yards']
        tourny_course_num = tournsum_data['years'][0]['tours'][0]['trns'][0]['courses'][e]['courseNum']
        if 'stimp' in leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0].keys():
            tourny_stimp = leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0]['stimp']
        else:
            tourny_stimp = 'no_stimp_reading'

        try:
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'][e].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        except:      
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course']['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course']['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course']['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        tourny_course_loc = 'no_course_loc_info'
        for course in course_loc_data['courses']:
            if course['courseNum'] == tourny_course_num:
                tourny_course_loc = course

        temp_dict = {'course_name': tourny_course_name, 'par_front': \
            tourny_par_front, 'par_back': tourny_par_back, 'course_yards': tourny_yardage,
            'stimp': tourny_stimp, 'grass_type': {'fairway_grass': tourny_grass_fairway, 'green_grass': tourny_grass_green,
            'rough_grass': tourny_grass_rough}, 'course_loc': tourny_course_loc, 'course_num': tourny_course_num}

        # set default key as list, and append all courses to the course key
        tourny_course_dict['_num_courses'] = str(len(course_data['courses']))  
        tourny_course_dict.setdefault('course', []).append(temp_dict)

    tournament_dict[tourny_code]['course_info'] = tourny_course_dict
    

    # add field info
    field_dict = {}
    for s in setup2_data['trn']['field']:
        player_id = s['id']
        player_first_name = s['name']['first']
        player_last_name = s['name']['last']
        player_ytd_money_rank = s['money']['ytdRank']
        player_current_country = s['info']['country']
        player_country_birth = s['info']['countryOfBirth']
        player_hometown = s['info']['hometown']
        player_status = s['status']
        field_dict[player_id] = {'first_name': player_first_name, 'last_name': player_last_name,
               'ytd_money_rank': player_ytd_money_rank, 'current_country': player_current_country, 
               'birth_country': player_country_birth, 'hometown': player_hometown, 'playing_status': player_status}
    
    tournament_dict[tourny_code]['tee_data'] = tee_data['tournament']
    tournament_dict[tourny_code]['field'] = field_dict


    tourny_start_date = tee_data['tournament']['rounds'][0]['courses'][0]['segments'][0]['groups'][0]['StartDate']
    tournament_dict[tourny_code]['start_date'] = tourny_start_date

    # Add weather dictionary
    api = 'c7c2d4dea244baeca35b91ed97364062'
    weather_dict = {}
    for d in range(4):
        lat = tourny_course_loc['latitude']
        lon = tourny_course_loc['longitude']
        date_object = datetime.datetime.strptime(tourny_start_date, '%m/%d/%Y')
        date_string = date_object + datetime.timedelta(days = d)
        weather = forecastio.load_forecast(api, lat, lon, time = date_string, units = 'us')
        hourly = weather.hourly()

        temp_weather_ls = []
        for hour in hourly.data:
            h = hour.d
            h['time_str'] = datetime.datetime.fromtimestamp(int(h['time'])).strftime('%m/%d/%Y %H:%M')
            temp_weather_ls.append(h)

        weather_dict[d] = temp_weather_ls

    tournament_dict[tourny_code]['weather'] = weather_dict

    base_path = '/home/valesco/Datasets/PGA_Data/PGA_Dir'
    os.chdir(base_path)

    make_path = base_path + '/' + tourny_prefix
    #os.mkdir(make_path)
    os.chdir(make_path)

    #file_name = tourny_prefix + '_course_data1.json'
    #with open(file_name, 'w') as outfile:
        #json.dump(tournament_dict, outfile)

    print(tourny_prefix, ' succeeded!!')
    #suceed_ls.append(tourny)


In [ ]:
### Single run through player round data

os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/')

tournaments = os.walk(os.getcwd())

tourny_dir = []

for tourny in tournaments:
    if len(tourny[1]) > 2:
        for t in tourny[1]:
            tourny_dir.append(t)
            

succeed_ls = []
failed_ls = []

tourny_dir = ['002_2016']
for tourny in tourny_dir:
    

    tourny_path = os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/' + tourny)
    tournament_json = open(tourny + '_course_data1.json')
    tournament_dict = json.load(tournament_json)

    tourny_number = tourny.split('_')[0]

    courses = int(tournament_dict[tourny_number]['course_info']['_num_courses'])

    player_list = tournament_dict[tourny_number]['field'].keys()

    tee_time_df = pd.DataFrame(columns = ['player_id', 'player_name',
                                          'tournament_id', 'course_name', 'course_num', 'time_zone', 'round_id', 'year',
                                          'round','date', 'tee_time', 'tee_segment', 'num_players_group',
                                          'vs_player1', 'vs_player2', 'vs_player3',
                                          'weather_visib', 'weather_precip_possib', 'weather_icon', 
                                          'weather_wind_speed', 'weather_dew_point', 'weather_temp', 
                                          'weather_pressure', 'weather_apparent_temp', 'weather_summary', 
                                          'weather_precip_inten', 'weather_cloud_cover', 'weather_wind_bearing',
                                          'weather_humidity', 'weather_time_str'])
    count = 0

    for player in player_list:
        for r, rnd in enumerate(tournament_dict[tourny_number]['tee_data']['rounds']):
            for course in rnd['courses']:
                course_name = course['CourseName']
                course_num = course['CourseNumber']
                #time_zone = tournament_dict[tourny_number]['course_info']['course'][0]['course_loc']['timezone']
                time_zone = 'America/Los_Angeles'
                if len(course_num) == 1:
                    course_num = '00' + course_num
                if len(course_num) == 2:
                    course_num = '0' + course_num

                for segment in course['segments']:
                    tee_segment = segment['SegmentID']

                    for group in segment['groups']:
                        player_id_ls = []
                        for player2 in group['players']:
                            if player == player2['TournamentPlayerID']:
                                tee_time = group['StartTime']
                                tee_date = group['StartDate']
                                for player3 in group['players']:
                                    if player != player3['TournamentPlayerID']:
                                        player_id_ls.append(player3['TournamentPlayerID'])

                                num_players_in_group = len(player_id_ls)

                                player_vars = ['vs_player1', 'vs_player2', 'vs_player3']

                                for v, var in enumerate(player_vars):
                                    try:
                                        globals()[var] = player_id_ls[v]
                                    except:
                                        globals()[var] = 'null'


                                name = player2['LFName']
                                round_id = '{}_{}_{}_{}'.format(player, year, tournament, r + 1)


                                #Create time string for weather

                                temp_time = tee_time.split(':')
                                hour = int(temp_time[0])
                                minute = int(temp_time[1].split(' ')[0])

                                if int(minute) > 29:
                                    hour = hour + 1
                                else:
                                    hour = hour

                                if 'pm' in tee_time:
                                    hour = hour + 12

                                minute = '00'
                                hour = str(hour)

                                if len(hour) < 2:
                                    hour = '0' + hour

                                weather_time = '{}:{}'.format(hour, minute)
                                weather_str = tee_date + ' ' + weather_time

                                for tourny_dict in tournament_dict[tourny_number]['weather'].items():
                                    for t_dict in tourny_dict[1]:
                                        if t_dict['time_str'] == weather_str:
                                            visib = t_dict['visibility']
                                            precip = t_dict['precipProbability']
                                            icon = t_dict['icon']
                                            wind_speed = t_dict['windSpeed']
                                            dew_point = t_dict['dewPoint']
                                            temp = t_dict['temperature']
                                            pressure = t_dict['pressure']
                                            app_temp = t_dict['apparentTemperature']
                                            summary = t_dict['summary']
                                            precip_int = t_dict['precipIntensity']
                                            cloud = t_dict['cloudCover']
                                            wind_bearing = t_dict['windBearing']
                                            humidity = t_dict['humidity']



                                tee_time_df.loc[count] = [player, name, tourny_number, course_name, course_num, time_zone,
                                            round_id, year, r + 1, tee_date, tee_time, tee_segment,
                                            num_players_in_group, vs_player1, vs_player2, vs_player3, visib, precip,
                                            icon, wind_speed, dew_point, temp, pressure, app_temp, 
                                            summary, precip_int, cloud, wind_bearing, humidity, weather_str]

                                count += 1

    file_name = tourny + '_player_round_data.csv'
    tee_time_df.to_csv(file_name)

    print(tourny, ' succeeded!!')
    succeed_ls.append(tourny)
